# explore gold and silver using FRED data

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")
fred<-Fred()

first, lets check what kind of data is available on the FRED for gold and silver

In [ ]:
goldFred <- fred$Meta() %>% inner_join(fred$TimeSeries(), by = 'SERIES_ID') %>%
    filter(NAME %like% '%gold%') %>%
    group_by(SERIES_ID, TICKER, NAME) %>%
    summarize(MIN_TS = min(TIME_STAMP), MAX_TS = max(TIME_STAMP)) %>%
    select(SERIES_ID, TICKER, NAME, MIN_TS, MAX_TS)

print(data.frame(goldFred))

In [ ]:
silverFred <- fred$Meta() %>% inner_join(fred$TimeSeries(), by = 'SERIES_ID') %>%
    filter(NAME %like% '%silver%') %>%
    group_by(SERIES_ID, TICKER, NAME) %>%
    summarize(MIN_TS = min(TIME_STAMP), MAX_TS = max(TIME_STAMP)) %>%
    select(SERIES_ID, TICKER, NAME, MIN_TS, MAX_TS)

print(data.frame(silverFred))

it looks like FRED only has data on silver from 2017. So lets focus on gold for now.

In [ ]:
seriesId <- -2147252004
goldDf <- data.frame(fred$TimeSeries() %>% filter(SERIES_ID == seriesId, TIME_STAMP >= as.Date('1995-01-01')) %>% select(TIME_STAMP, VAL))
goldDf$TIME_STAMP <- as.Date(goldDf$TIME_STAMP)

print(nrow(goldDf))
print(head(goldDf))

In [ ]:
firstDate <- min(goldDf$TIME_STAMP)
lastDate <- max(goldDf$TIME_STAMP)
xAxisTicks <- seq(from=firstDate, to=lastDate, length.out=10)

fredDetails <- data.frame(goldFred %>% filter(SERIES_ID == seriesId))

options(repr.plot.width=16, repr.plot.height=8)

ggplot(goldDf, aes(x=TIME_STAMP, y=VAL)) +
    theme_economist() +
    #theme(text = element_text(size = 3), element_line(size = 0.1)) +
    geom_line() +
    scale_y_log10() +
    scale_x_date(breaks = xAxisTicks) +
    labs(x='', y='log(price)', color='', title=sprintf("FRED: %s - %s", fredDetails$TICKER[1], fredDetails$NAME[1]), 
         subtitle=sprintf("[%s:%s]", firstDate, lastDate)) +
    annotate("text", x=lastDate, y=min(goldDf$VAL, na.rm=T), label = "@StockViz", hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)